In [1]:
import numpy as np
import tensorflow as tf
import word_preprocessing as wp

[nltk_data] Downloading package punkt to /home/kaushik/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
ml_classes = ['location', 'professor', 'time']
processor  =  wp.Word_Processor('../grammar/')
feature_set,class_set,lexicon_size = processor.words_to_vectors()


In [3]:
lstm_units = 128
batch_size = 8
lstm_layers = 1
learning_rate = 0.001

def split_sets(data, classes ,split_point = 1):
    split_length  = int(data.shape[0] * split_point)
    training_features = data[:split_length]
    training_ops = classes[:split_length]
    testing_features = data[split_length:]
    testing_ops = classes[split_length:]
    return training_features,training_ops,testing_features,testing_ops

train_x,train_y,test_x,test_y = split_sets(feature_set, class_set)


In [4]:
outputs = tf.placeholder(tf.int64 , [batch_size, 1])
y  = tf.one_hot(indices=outputs,depth=len(ml_classes))
inputs = tf.placeholder(tf.float64, [batch_size, lexicon_size, 1], name="inputs")


def create_lstm_network(batch_size,inputs):
    def create_lstm_cell(units):
        lstm  = tf.contrib.rnn.BasicLSTMCell(units)
        return tf.contrib.rnn.DropoutWrapper(lstm, output_keep_prob=1)

    def create_stacked_lstm(layers, units):
        return tf.contrib.rnn.MultiRNNCell([create_lstm_cell(units) for _ in range(layers)])
    
    cell =  create_lstm_cell(lstm_units)
    inits = cell.zero_state(batch_size,tf.float64)
    lstm_outputs,states =  tf.nn.dynamic_rnn(cell, inputs, dtype=tf.float64, initial_state=inits)
    predictions = tf.contrib.layers.fully_connected(lstm_outputs[:, -1], num_outputs= len(ml_classes), activation_fn=tf.nn.relu)
    return predictions

In [5]:
pred = create_lstm_network(batch_size,inputs)
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels = y,logits=pred))
tf.summary.scalar('cost', cost)
optimizer = tf.train.AdamOptimizer(learning_rate = learning_rate).minimize(cost)
tf.summary.histogram('predictions', pred)
merged = tf.summary.merge_all()

In [6]:
def get_batches(x, y, batch_size=100):
    n_batches = len(x)//batch_size
    x, y = x[:n_batches*batch_size], y[:n_batches*batch_size]
    for ii in range(0, len(x), batch_size):
        yield np.array(x[ii:ii+batch_size]), np.array(y[ii:ii+batch_size])

In [7]:
epochs = 10
from tensorflow.python import debug as tf_debug

saver = tf.train.Saver()
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
   # sess = tf_debug.LocalCLIDebugWrapperSession(sess)
   # sess.add_tensor_filter("has_inf_or_nan", tf_debug.has_inf_or_nan)
    train_writer = tf.summary.FileWriter('./logs/lstm-chatbot/train', sess.graph)
    test_writer = tf.summary.FileWriter('./logs/lstm-chatbot/test', sess.graph)
    iteration = 1
    for epoch in range(epochs):
        #state = sess.run(initial_state)
        for ii, (x, y) in enumerate(get_batches(train_x, train_y, batch_size), 1):
            x  =  np.reshape(x, newshape = [batch_size ,lexicon_size, 1])
            y =  np.reshape(y, newshape=[batch_size,1])
            feed = {inputs: x,outputs : y}
            #sess.run(outputs, feed_dict=feed)
            #print(outputs.eval(feed_dict=feed))
            pre,summary, loss, _ = sess.run([pred,merged, cost, optimizer], feed_dict=feed)
            train_writer.add_summary(summary, iteration)
        
            if iteration%100 == 0:
                print("Epoch: {}/{}".format(epoch, epochs),
                      "Iteration: {}".format(iteration),
                      "Train loss: {:.3f}".format(loss))
                
                print("Expected, ",y)
                print("Predicted , ", pre)
                
            iteration +=1
     
            
    saver.save(sess, "checkpoints/lstm_chatbot.ckpt")
 
    
    
    

Epoch: 0/10 Iteration: 100 Train loss: 0.819
Expected,  [[0]
 [2]
 [0]
 [2]
 [2]
 [2]
 [2]
 [2]
 [0]
 [2]]
Predicted ,  [[ 0.          0.          1.58612375]
 [ 0.          0.          1.5860947 ]
 [ 0.          0.          1.58612433]
 [ 0.          0.          1.5860932 ]
 [ 0.          0.          1.588214  ]
 [ 0.          0.          1.58609384]
 [ 0.          0.          1.58516058]
 [ 0.          0.          1.58475367]
 [ 0.          0.          1.58612496]
 [ 0.          0.          1.58579207]]
Epoch: 1/10 Iteration: 200 Train loss: 0.541
Expected,  [[2]
 [2]
 [2]
 [2]
 [2]
 [2]
 [2]
 [2]
 [2]
 [0]]
Predicted ,  [[ 0.          0.          1.98500072]
 [ 0.          0.          0.81730928]
 [ 0.          0.          2.09340466]
 [ 0.          0.          1.83669706]
 [ 0.          0.          1.07204429]
 [ 0.          0.          1.60973445]
 [ 0.          0.          0.33896833]
 [ 0.          0.          0.46023455]
 [ 0.          0.          1.4851292 ]
 [ 0.          0. 

KeyboardInterrupt: 

In [ ]:
test_input = tf.placeholder(tf.float64, [test_batch, lexicon_size, 1], name="test_inputs")
test_prediction = create_lstm_network(test_batch, test_input)
with tf.Session() as sess:
    test_batch = 1
    #test_prediction = create_lstm_network(test_batch, test_input)
    sess.run(tf.global_variables_initializer())
    saver =  tf.train.Saver()
    # = tf.train.import_meta_graph('checkpoints/lstm_chatbot.ckpt.meta')
    saver.restore(sess,tf.train.latest_checkpoint('checkpoints/'))
    tr_data = processor.vectorize_input("hi who is professor for class")
    tr_data = np.reshape(tr_data, newshape=[test_batch, lexicon_size , 1])
    pred_class = sess.run(test_prediction, feed_dict={test_input:tr_data})
    print(pred_class)
    for var in tf.trainable_variables():
        print(var.name)
            
      









    